<a href="https://colab.research.google.com/github/Yas12Hatsu/reto28_Hackaton/blob/main/reto28_Hackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install langchain pypdf openai chromadb tiktoken

Agrega el API KEY

In [ ]:
from getpass import getpass
import os

OPENAI_API_KEY=getpass("Agrega tu KEY de OpenIA: ")
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

Agrega tu KEY de OpenIA: ··········


In [ ]:
import requests
from langchain.document_loaders import PyPDFLoader

urls = [
    'https://uttecamac.edomex.gob.mx/sites/uttecamac.edomex.gob.mx/files/files/PIDE-2020-2024.pdf',
]

pdfdoc = []

for i, url in enumerate(urls):
  response = requests.get(url)
  filename = f'paper{i+1}.pdf'
  with open(filename, "wb") as f:
    f.write(response.content)
    print(f"{filename} fue descargado con éxito.")

    loader = PyPDFLoader(filename)
    data = loader.load()
    pdfdoc.extend(data)

print('Contenido de pdf: ')
print()


paper1.pdf fue descargado con éxito.
Contenido de pdf: 



In [ ]:
type(pdfdoc), len(pdfdoc), pdfdoc[3]

(list,
 187,
 Document(page_content='UNIVERSIDAD  TECNOLÓGICA  DE TECÁMAC  \n4 PROGRAMA  INSTITUCIONAL  DE DESARROLLO  2020 - 2024   \n  \nREPRESENTANTES  DEL SECTOR  PRODUCTIVO  \n \n \nQ.F.B.  José  Raúl Flores  Fletes  \nDirector  General  Biograft  de México  S.A de C.V. \n \n \nLcdo.  Mauricio  Núñez  Avendaño  \nPresidente  de COPARMEX  Oriente  del Estado  de México  \n \n \n \n \nREPRESENTANTE  DE LA SECRETARÍA  DE LA CONTRALORÍA  \n \n \nTITULAR  SUPLENTE  \n \nC.P. Emilio  Eduardo  Ruiz  Nonato  \nDirector de Control y Evaluación “A -I” de la  \nSecretaría  de Contraloría   \nC.P. Joel  Mora  López  \nTitular del Órgano Interno de Control de la  \nUniversidad  Tecnológica  de Tecámac  \n \n \n \nSECRETARIO  \n \n \n \nM. en C. Rafael  Adolfo  Núñez  González  \nRector  de la  \nUniversidad  Tecnológica  de Tecámac  ', metadata={'source': 'paper1.pdf', 'page': 3}))

Split de los documentos

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
    length_function = len
)

documents = text_splitter.split_documents(pdfdoc)

In [ ]:
len(documents), documents[10]

(329,
 Document(page_content='UNIVERSIDAD  TECNOLÓGICA  DE TECÁMAC  \n9 PROGRAMA  INSTITUCIONAL  DE DESARROLLO  2020 - 2024   \n  \nPRESENTACIÓN  \n \nEl Programa  Institucional  de Desarrollo  2020 -2024,  fue elaborado  con entusiasmo  al \ninterior  de la Universidad  Tecnológica  de Tecámac,  como  un ejercicio  de planeación  \nestratégica y participativa a mediano plazo, contiene un conjunto de políticas, acciones  \nestratégicas y diferentes procesos administrativos para orientar el desarrollo armónico de  \nlas funciones sustantivas que realizan las diferentes áreas y  las buenas prácticas de otras  \nuniversidades,  proponiendo panoramas  deseables y  posibles  de alcanzar.  \n \nEn este ejercicio de planeación estratégica de la institución se establecieron objetivos,  \nmetas,  acciones  estratégicas,  prioridades  y se fijaron  responsabilidades  y tiempos  de \nejecución, además de d ar seguimiento a todas las acciones emprendidas y evaluar los  \nresultados periódicamente 

Base de datos vectorial

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

vectorstore = Chroma.from_documents(
    documents = documents,
    embedding = embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs = {"k" : 2}
)

Modelos de chat y cadenas para consultar de infromación

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key = OPENAI_API_KEY,
    model_name = "gpt-3.5-turbo",
    temperature = 0.0,
)

qa_chain = RetrievalQA.from_chain_type(
    llm = chat,
    chain_type = "stuff",
    retriever = retriever,
)

In [ ]:
query = "¿Cuántos programas educativos de nivel 5A están relacionados con el área de ingeniería en la Universidad Tecnológica de Tecámac? '¿Cuáles son?, ¿Existe Ingeniería en Tecnologías de la Información?"
qa_chain.run(query)

'En la Universidad Tecnológica de Tecámac, hay un total de 6 programas educativos de nivel 5A relacionados con el área de ingeniería. Los programas son:\n\n1. Ingeniería en Biotecnología\n2. Ingeniería en Energías Renovables\n3. Ingeniería en Gestión de Proyectos\n4. Ingeniería en Mantenimiento Industrial\n5. Ingeniería en Mecatrónica\n6. Ingeniería en Nanotecnología\n\nSí, existe el programa educativo de Ingeniería en Tecnologías de la Información y Comunicación en la Universidad Tecnológica de Tecámac.'